# 🎧 Emotion Prediction Notebook

This notebook allows you to test the trained model on custom `.wav` files and view the predicted emotion output.


In [5]:
# predict_emotion.py

import sys
import numpy as np
import joblib
import librosa

# Load model and label encoder
model = joblib.load("model/trained_model.pkl")
label_encoder = joblib.load("model/label_encoder.pkl")

# Feature extraction functions
def mfcc_values(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)

def delta_values(filename):
    mfcc = mfcc_values(filename)
    return librosa.feature.delta(mfcc)

def log_mel_values(file_path, duration=3, offset=0.5, n_mels=128):
    y, sr = librosa.load(file_path, duration=duration, offset=offset)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    return np.mean(librosa.power_to_db(mel_spec).T, axis=0)

def zcr_values(file_name):
    sig, sr = librosa.load(file_name, duration=3, offset=0.5)
    return np.mean(librosa.feature.zero_crossing_rate(sig).T, axis=0)

def spectral_features(file_path, duration=3, offset=0.5):
    y, sr = librosa.load(file_path, duration=duration, offset=offset)
    centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)
    rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    flatness = np.mean(librosa.feature.spectral_flatness(y=y))
    return np.hstack([centroid, bandwidth, rolloff, flatness, contrast])

def extract_features(file_name):
    return np.hstack([
        mfcc_values(file_name),
        delta_values(file_name),
        log_mel_values(file_name),
        zcr_values(file_name),
        spectral_features(file_name)
    ])



audio_path = "enter test file path "  # hardcode or use input()
features = extract_features(audio_path).reshape(1, -1)

# Predict emotion
prediction = model.predict(features)
emotion = label_encoder.inverse_transform(prediction)[0]
print(f"🎧 Predicted Emotion: {emotion}")


🎧 Predicted Emotion: neutral


## ✅ How to Use

1. Place your test `.wav` file in the same directory or provide full path.
2. Run all cells to extract features and predict emotion.
3. You will see the predicted emotion printed as output.
